# Logistic Regression Titanic

In [31]:
from pyspark.sql import SparkSession

In [32]:
spark = SparkSession.builder.appName('titanic').getOrCreate()

In [33]:
data = spark.read.csv('../data/titanic.csv',inferSchema=True,header=True)

In [36]:
data.show(n=5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [34]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [37]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [38]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [39]:
my_final_data = my_cols.na.drop()

In [40]:
my_final_data.show()

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
|       0|     3|  male|35.0|    0|    0|   8.05|       S|
|       0|     1|  male|54.0|    0|    0|51.8625|       S|
|       0|     3|  male| 2.0|    3|    1| 21.075|       S|
|       1|     3|female|27.0|    0|    2|11.1333|       S|
|       1|     2|female|14.0|    1|    0|30.0708|       C|
|       1|     3|female| 4.0|    1|    1|   16.7|       S|
|       1|     1|female|58.0|    0|    0|  26.55|       S|
|       0|     3|  male|20.0|    0|    0|   8.05|       S|
|       0|     3|  male|39.0|    1|    5| 31.275|       S|
|       0|     3|female|14.0|    0|    0| 7.8542|       

### Working with Categorical Columns

Let's break this down into multiple steps to make it all clear.

In [41]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [42]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [43]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [44]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [45]:
from pyspark.ml.classification import LogisticRegression

## Pipelines 

Let's see an example of how to use pipelines (we'll get a lot more practice with these later!)

In [46]:
from pyspark.ml import Pipeline

In [47]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [48]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [49]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [50]:
fit_model = pipeline.fit(train_titanic_data)

In [51]:
results = fit_model.transform(test_titanic_data)

In [54]:
results.show(n = 5, vertical=True)

-RECORD 0-----------------------------
 Survived      | 0                    
 Pclass        | 1                    
 Sex           | female               
 Age           | 25.0                 
 SibSp         | 1                    
 Parch         | 2                    
 Fare          | 151.55               
 Embarked      | S                    
 SexIndex      | 1.0                  
 EmbarkIndex   | 0.0                  
 SexVec        | (1,[],[])            
 EmbarkVec     | (2,[0],[1.0])        
 features      | [1.0,0.0,25.0,1.0... 
 rawPrediction | [-2.5839176475747... 
 probability   | [0.07018065263515... 
 prediction    | 1.0                  
-RECORD 1-----------------------------
 Survived      | 0                    
 Pclass        | 1                    
 Sex           | male                 
 Age           | 22.0                 
 SibSp         | 0                    
 Parch         | 0                    
 Fare          | 135.6333             
 Embarked      | C       

In [55]:
results.select("Survived", "prediction").show(n=30)

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 30 rows



In [56]:
results.show(n = 15, vertical=True)

-RECORD 0-----------------------------
 Survived      | 0                    
 Pclass        | 1                    
 Sex           | female               
 Age           | 25.0                 
 SibSp         | 1                    
 Parch         | 2                    
 Fare          | 151.55               
 Embarked      | S                    
 SexIndex      | 1.0                  
 EmbarkIndex   | 0.0                  
 SexVec        | (1,[],[])            
 EmbarkVec     | (2,[0],[1.0])        
 features      | [1.0,0.0,25.0,1.0... 
 rawPrediction | [-2.5839176475747... 
 probability   | [0.07018065263515... 
 prediction    | 1.0                  
-RECORD 1-----------------------------
 Survived      | 0                    
 Pclass        | 1                    
 Sex           | male                 
 Age           | 22.0                 
 SibSp         | 0                    
 Parch         | 0                    
 Fare          | 135.6333             
 Embarked      | C       

In [57]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [59]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [60]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [61]:
AUC = my_eval.evaluate(results)

In [62]:
AUC

0.7736905237904839